In [133]:
# Nous chargeons d'abord toutes les libraires que nous allons utiliser
import pickle
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence

In [134]:
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn import grid_search
import xgboost
import numpy as np
from keras import layers, models, optimizers
from keras.models import Sequential

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [135]:
from nltk.corpus import stopwords
from nltk import RegexpTokenizer,WhitespaceTokenizer

## Chargeons les donnees 

In [136]:
train_seq = pd.read_pickle('../input/Data/Learn/sequences.pkl')
train_snt = pd.read_pickle('../input/Data/Learn/sentences.pkl')
train_labels = pd.read_pickle('../input/Data/Learn/labels.pkl')
dfdict = pd.read_pickle('../input/Data/dict.pkl')

In [137]:
test_seq = pd.read_pickle('../input/Data/Test/sequences_t.pkl')
test_snt = pd.read_pickle('../input/Test/sentences_t.pkl')

In [138]:
tok2 = RegexpTokenizer(r'''(?x)
          \d+(\.\d+)?\s*%   # les pourcentages
        | \w'               # les contractions d', l', ...
        | \w+               # les mots pleins
        | [^\w\s]           # les ponctuations
        ''')
toknizer = RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')

In [139]:
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()

In [140]:

def stem_sentences(d):
    tokens = toknizer.tokenize(d)
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return (' ').join(stemmed_tokens)

def tokenize_arr(dt):
    newdt = []
    for d in dt:
        newdt.append(stem_sentences(d))
    return newdt




In [141]:
train_snt_n = tokenize_arr(train_snt)

In [143]:
lbl_enc = preprocessing.LabelEncoder()
train_labels_enc = lbl_enc.fit_transform(train_labels)

In [ ]:
# Création du dataset de training et de validation sur les phrases
train_snt_x, valid_snt_x, train_y, valid_y = train_test_split(train_snt_n, train_labels_enc, 
                                                  stratify=train_labels_enc, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [ ]:
# Création du dataset de training et de validation sur les sequences
train_seq_x, valid_seq_x, train_seq_y, valid_seq_y = train_test_split(train_seq, train_labels_enc, 
                                                  stratify=train_labels_enc, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [ ]:

sum =0
for lab in train_labels_enc:
    sum += lab
print(sum)
sum/(len(train_labels))
n_mitterrand = sum
n_chirac = len(train_labels) - sum

In [ ]:
sum =0
for lab in train_y:
    sum += lab
n_mitterrand_train = sum
n_chirac_train = len(train_y) - n_mitterrand_train

In [ ]:
print(len(train_snt_x))
print(len(train_y))
print(n_mitterrand)
print(n_mitterrand_train)
print(n_chirac_train)

In [ ]:
index = 0
train_snt_x_reduced = []
train_seq_x_reduced = []
train_y_reduced = []
compteur = 0
for t in train_y:
    print(compteur)
    
    if(train_y[index]==0):
        if(compteur<n_mitterrand_train):
            train_y_reduced.append( train_y[index])
            train_snt_x_reduced.append(train_snt_x[index])
            train_seq_x_reduced.append(train_seq_x[index])
            compteur +=1
    else:
        train_snt_x_reduced.append(train_snt_x[index])
        train_y_reduced.append(train_y[index])
        train_seq_x_reduced.append(train_seq_x[index])
    index+=1

In [ ]:
# les données ne sont pas équilibrées entre les classes ´Chirac´ et ´Mitterrand´; je réalise une extension des données de training pour ré-equilibrer.

index = 0
train_snt_x_extended = []
train_seq_x_extended = []
train_y_extended = []


train_snt_x_mitterand = []
train_seq_x_mitterand = []
for t in train_y:
    
    if(train_y[index]==1):
        train_snt_x_mitterand.append(train_snt_x[index])
        train_seq_x_mitterand.append(train_seq_x[index])
    train_snt_x_extended.append(train_snt_x[index])
    train_seq_x_extended.append(train_seq_x[index])
    train_y_extended.append(t)
    index+=1

compteur = 0 
while(compteur <(n_chirac_train - n_mitterrand_train)):
    index = compteur % n_mitterrand_train
    train_snt_x_extended.append(train_snt_x_mitterand[index])
    train_seq_x_extended.append(train_seq_x_mitterand[index])
    train_y_extended.append(1)
    compteur+=1

# **Etape 1 : Feature engeneering**

In [ ]:
#Feature 1 :  Objet CountVectorize
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_snt_x_extended + valid_snt_x)

train_xcount_aumented =  count_vect.transform(train_snt_x_extended)
valid_xcount_aumented =  count_vect.transform(valid_snt_x)
test_xcount_aumented = count_vect.transform(test_snt)

In [ ]:
#Feature 2 : TF_IDF

tfidf_vect = TfidfVectorizer(sublinear_tf=True, max_df=0.5, analyzer='word', 
            vocabulary=dfdict,lowercase=False)

tfidf_vect.fit(list(train_snt_x_extended) + list(valid_snt_x))
train_snt_x_vect =  tfidf_vect.transform(train_snt_x_extended) 
valid_snt_x_vect = tfidf_vect.transform(valid_snt_x)
test_snt_vect = tfidf_vect.transform(test_snt)


# Word embedding
 Nous chargeons un modele pre-trained de Fasttext que Facebook fourni pour le francais, nous l´utiliserons pour un layer de nos modeles de neural network

In [145]:
# Nous chargeons le matrices pré entraines sur un corpus grand
embeddings_index = {}
for i, line in enumerate(open('../input/deeplearningtest/wiki.fr.vec')):
    values = line.strip().split(' ')
    #print(values)
    try:
        embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')
    except:
        print(values)
        input()

In [146]:
maxlen = 0
for t in train_seq_x_aumented:
    if(maxlen<len(t)):
        maxlen = len(t)
    
print(maxlen)

437


In [147]:
#"Pad" les vecteurs pour une longueur egale
train_seq_x_pad = sequence.pad_sequences(train_seq_x_aumented, maxlen=437)
valid_seq_x_pad = sequence.pad_sequences(valid_seq_x, maxlen=437)

In [148]:
# Nous créons la matrices embedding pour le dictionnaire que nous avons dans nos discours
embedding_matrix = np.zeros((len(dfdict) + 1, 300))
for word, i in dfdict.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Etape 2 : Preparation des modèles de classification

In [ ]:
# Définition de la fonction de perte
def multiclass_logloss(actual, predicted, eps=1e-15):
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [ ]:
# Définition de la fonction d'évalutation du modèle
# Les données étant déséquilibrées je vais privilégier une évaluation de la metrique recall - precision F1 plutot qu'une simple 'accuracy'
def resultModel(mod,xtrain,xvalid,ytrain,yvalid):
    mod.fit(xtrain, ytrain)
    predictions = mod.predict(xvalid)
    predictions_proba = mod.predict_proba(xvalid)
    #print(predictions)
    print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions_proba))
    print("accuracy :" + str(metrics.accuracy_score(predictions, yvalid)))
    print("recall :" + str(metrics.recall_score(predictions, yvalid)))
    print("precision :" + str(metrics.precision_score(predictions, yvalid)))
    f1 = 2 * (metrics.precision_score(predictions, yvalid)*metrics.recall_score(predictions, yvalid)/(metrics.recall_score(predictions, yvalid) + metrics.precision_score(predictions, yvalid)))
    print("F1 :" + str(f1))
    return mod

# Benchmark de modèle

In [ ]:
# Regression linéaire
print("LR avec vector count")
modLR_v = resultModel(LogisticRegression(C=1.0),train_xcount,valid_xcount,train_y_aumented,valid_y)
print("LR avec tfidf")
modLR_tfidf = resultModel(LogisticRegression(C=1.0),train_snt_x_vect,valid_snt_x_vect,train_y_aumented,valid_y)



In [ ]:
# Naive Bayes
print("NB avec vector count")
resultModel(MultinomialNB(),train_xcount,valid_xcount,train_y_aumented,valid_y)
print("NB avec tfidf")
resultModel(MultinomialNB(),train_snt_x_vect,valid_snt_x_vect,train_y_aumented,valid_y)

In [ ]:
#SVM avec parametres fixes
print("SVM avec vector count")
resultModel(SVC(C=1.0, probability=True),train_xcount_aumented,valid_xcount_aumented,train_y_aumented,valid_y)
print("SVM avec tfidf")
resultModel(SVC(C=1.0, probability=True),train_snt_x_vect,valid_snt_x_vect,train_y,valid_y)

### Pour améliorer le modèle SVM nous pourrions réaliser une recherche de paramètres optimaux avec une gridsearch

Je vais exporter les résultats du modèle linéaire qui me donne les meilleurs performances

In [149]:
predictionsLR_encode = modLR_v.predict(test_xcount)
predictionsLR_decode = lbl_enc.inverse_transform(predictionsLR_encode)
predictionsLR_decode.tolist()

with open('labels_lr.pkl', 'wb') as handle:
    pickle.dump(predictionsLR_decode, handle, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'modLR_v' is not defined

# Etape 3 : Elaboration d'un modèle de Deep Learning

In [ ]:
def resultModelDeep(mod,xtrain,xvalid,ytrain,yvalid):
    mod.fit(xtrain, ytrain,batch_size=1024, epochs=1,validation_data=(xvalid, yvalid))
    predictions = mod.predict(xvalid).argmax(axis=-1)
    print("accuracy :" + str(metrics.accuracy_score(predictions, yvalid)))
    print("recall :" + str(metrics.recall_score(predictions, yvalid)))
    print("precision :" + str(metrics.precision_score(predictions, yvalid)))
    f1 = 2 * (metrics.precision_score(predictions, yvalid)*metrics.recall_score(predictions, yvalid)/(metrics.recall_score(predictions, yvalid) + metrics.precision_score(predictions, yvalid)))
    print("F1 :" + str(f1))

In [ ]:
def cnn():
    input_layer = layers.Input((437, ))

    # Le premier "Layer" est celui du word embedding dont on a chargé la matrice
    embedding_layer = layers.Embedding(len(dfdict) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Hidden Layers Dense
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="softmax")(output_layer1)
    
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model


In [ ]:
# Avant d'entrainer le modèle nous allons scaler les données
scl = preprocessing.StandardScaler()
train_seq_x_pad = scl.fit_transform(train_seq_x_pad)
valid_seq_x_pad = scl.transform(valid_seq_x_pad)

In [ ]:
print("CNN, Word Embeddings")

resultModelDeep(cnn(), train_seq_x_pad,valid_seq_x_pad,np.asarray(train_y_aumented),valid_y)

Les resultats ne sont pas convaincants, une seule classe est predicte. Ou bien le pre-train word embedding ne peut pas servir a prédire le propriétaire du discours, ou bien il faut realiser une étude plus poussée sur les paramètres du modèle de neural network pour obtenir de meilleurs resultats.